## 🎯 特徵工程總結與下一步

### ✅ 已完成的特徵工程

1. **時間特徵** ⏰
   - 從 TransactionDT 提取小時、星期、週末標識
   - 時間段分類（夜晚、早晨、下午、晚上）
   - 發現詐騙交易的時間模式

2. **交易金額特徵** 💰
   - 對數和平方根變換處理偏斜分佈
   - 金額範圍分組（小額、中額、大額等）
   - 整數金額和常見金額標識
   - 不同金額範圍的詐騙風險分析

3. **聚合特徵** 📊
   - 基於卡片的歷史交易統計
   - 基於地址的行為聚合
   - 用戶歷史詐騙率和交易模式

### 🔍 主要發現

- **時間模式**: 特定時段的詐騙風險較高
- **金額特徵**: 不同金額範圍呈現不同的詐騙風險
- **聚合統計**: 歷史行為是強有力的詐騙指標
- **特徵相關性**: 新創建的特徵與詐騙標籤有顯著相關性

### 🔄 建議的下一步

1. **特徵選擇** 🎯
   - 使用統計方法篩選最重要特徵
   - 移除高相關性和低方差特徵
   - 基於業務邏輯驗證特徵有效性

2. **數據預處理** 🛠️
   - 處理缺失值和異常值
   - 特徵標準化和正規化
   - 類別不平衡處理（SMOTE等）

3. **模型訓練** 🤖
   - 嘗試多種機器學習算法
   - 超參數調優和交叉驗證
   - 模型融合和集成學習

4. **模型評估** 📈
   - ROC-AUC、PR-AUC等關鍵指標
   - 業務指標評估（精確率、召回率）
   - 模型可解釋性分析

---

**✨ 特徵工程階段完成！數據已準備好進入建模階段。**

In [ ]:
# 3. 聚合特徵工程
print("📊 創建聚合特徵...")

try:
    # 創建基於卡片的聚合特徵
    df_with_card = feature_engineer.create_card_based_features(df_with_amount)
    print("✅ 卡片聚合特徵創建完成")
    
    # 創建基於地址的聚合特徵  
    df_with_addr = feature_engineer.create_address_based_features(df_with_card)
    print("✅ 地址聚合特徵創建完成")
    
    # 統計新增特徵數量
    original_features = df.shape[1]
    final_features = df_with_addr.shape[1]
    new_features_count = final_features - original_features
    
    print(f"📈 特徵工程總結:")
    print(f"- 原始特徵數: {original_features}")
    print(f"- 最終特徵數: {final_features}")
    print(f"- 新增特徵數: {new_features_count}")
    
    # 特徵重要性分析（簡單相關性）
    print(f"\n🔍 分析新特徵與詐騙的相關性...")
    
    # 獲取數值型特徵
    numeric_features = df_with_addr.select_dtypes(include=[np.number]).columns
    target_correlations = df_with_addr[numeric_features].corrwith(df_with_addr['isFraud']).abs().sort_values(ascending=False)
    
    # 顯示前15個最相關的特徵
    print(f"\n📊 與詐騙最相關的前15個特徵:")
    display(target_correlations.head(15).to_frame('相關性'))
    
    # 視覺化特徵相關性
    plt.figure(figsize=(12, 8))
    top_features = target_correlations.head(20)
    plt.barh(range(len(top_features)), top_features.values)
    plt.yticks(range(len(top_features)), top_features.index)
    plt.xlabel('與詐騙的絕對相關性')
    plt.title('Top 20 特徵與詐騙相關性')
    plt.gca().invert_yaxis()
    plt.tight_layout()
    plt.show()
    
    # 保存處理後的數據
    df_final = df_with_addr
    print(f"\n💾 特徵工程完成，最終數據形狀: {df_final.shape}")
    
except Exception as e:
    print(f"❌ 聚合特徵創建失敗: {e}")
    df_final = df_with_amount  # 使用前一步的結果

In [ ]:
# 2. 交易金額特徵工程
print("💰 創建交易金額特徵...")

try:
    # 創建交易金額特徵
    df_with_amount = feature_engineer.create_transaction_amount_features(df_with_time)
    
    # 檢查新增的金額特徵
    amount_features = ['TransactionAmt_log', 'TransactionAmt_sqrt', 'amt_range', 'is_round_amount', 'is_common_amount']
    print(f"✅ 交易金額特徵創建成功，新增 {len(amount_features)} 個特徵")
    
    # 視覺化金額特徵分佈
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    # 原始金額分佈
    axes[0,0].hist(df_with_amount['TransactionAmt'], bins=50, alpha=0.7)
    axes[0,0].set_title('交易金額分佈（原始）')
    axes[0,0].set_xlabel('交易金額')
    axes[0,0].set_ylabel('頻率')
    axes[0,0].set_yscale('log')
    
    # 對數變換後分佈
    axes[0,1].hist(df_with_amount['TransactionAmt_log'], bins=50, alpha=0.7, color='orange')
    axes[0,1].set_title('交易金額分佈（對數變換）')
    axes[0,1].set_xlabel('log(交易金額+1)')
    axes[0,1].set_ylabel('頻率')
    
    # 金額範圍詐騙率
    amount_fraud = df_with_amount.groupby('amt_range')['isFraud'].agg(['count', 'mean']).reset_index()
    axes[1,0].bar(range(len(amount_fraud)), amount_fraud['mean'])
    axes[1,0].set_title('不同金額範圍的詐騙率')
    axes[1,0].set_xlabel('金額範圍')
    axes[1,0].set_ylabel('詐騙率')
    axes[1,0].set_xticks(range(len(amount_fraud)))
    axes[1,0].set_xticklabels(amount_fraud['amt_range'], rotation=45)
    
    # 整數金額 vs 詐騙率
    round_amount_fraud = df_with_amount.groupby('is_round_amount')['isFraud'].mean()
    axes[1,1].bar(['非整數', '整數'], round_amount_fraud.values)
    axes[1,1].set_title('整數金額 vs 詐騙率')
    axes[1,1].set_ylabel('詐騙率')
    
    plt.tight_layout()
    plt.show()
    
    print("💡 交易金額特徵洞察:")
    print(f"- 平均交易金額: ${df_with_amount['TransactionAmt'].mean():.2f}")
    print(f"- 中位數交易金額: ${df_with_amount['TransactionAmt'].median():.2f}")
    print(f"- 整數金額詐騙率: {round_amount_fraud[1]:.4f}")
    print(f"- 非整數金額詐騙率: {round_amount_fraud[0]:.4f}")
    print(f"- 最高風險金額範圍: {amount_fraud.loc[amount_fraud['mean'].idxmax(), 'amt_range']}")
    
except FeatureCreationError as e:
    print(f"❌ 交易金額特徵創建失敗: {e.message}")
except Exception as e:
    print(f"❌ 未預期錯誤: {e}")

In [ ]:
# 1. 時間特徵工程
print("🕒 創建時間特徵...")

try:
    # 創建時間特徵
    df_with_time = feature_engineer.create_time_features(df)
    
    # 檢查新增的時間特徵
    time_features = ['hour', 'day', 'week', 'time_of_day', 'is_weekend']
    print(f"✅ 時間特徵創建成功，新增 {len(time_features)} 個特徵")
    
    # 視覺化時間特徵分佈
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    # 按小時的詐騙率
    hourly_fraud = df_with_time.groupby('hour')['isFraud'].agg(['count', 'mean']).reset_index()
    axes[0,0].bar(hourly_fraud['hour'], hourly_fraud['mean'])
    axes[0,0].set_title('詐騙率按小時分佈')
    axes[0,0].set_xlabel('小時')
    axes[0,0].set_ylabel('詐騙率')
    
    # 按星期的詐騙率
    daily_fraud = df_with_time.groupby('day')['isFraud'].agg(['count', 'mean']).reset_index()
    axes[0,1].bar(daily_fraud['day'], daily_fraud['mean'])
    axes[0,1].set_title('詐騙率按星期分佈')
    axes[0,1].set_xlabel('星期')
    axes[0,1].set_ylabel('詐騙率')
    
    # 時間段分佈
    time_period_counts = df_with_time['time_of_day'].value_counts()
    axes[1,0].pie(time_period_counts.values, labels=time_period_counts.index, autopct='%1.1f%%')
    axes[1,0].set_title('交易時間段分佈')
    
    # 週末vs工作日
    weekend_fraud = df_with_time.groupby('is_weekend')['isFraud'].mean()
    axes[1,1].bar(['工作日', '週末'], weekend_fraud.values)
    axes[1,1].set_title('詐騙率：週末 vs 工作日')
    axes[1,1].set_ylabel('詐騙率')
    
    plt.tight_layout()
    plt.show()
    
    print("💡 時間特徵洞察:")
    print(f"- 平均每小時詐騙率: {hourly_fraud['mean'].mean():.4f}")
    print(f"- 最高風險時段: {hourly_fraud.loc[hourly_fraud['mean'].idxmax(), 'hour']}時")
    print(f"- 週末詐騙率: {weekend_fraud[1]:.4f}")
    print(f"- 工作日詐騙率: {weekend_fraud[0]:.4f}")
    
except FeatureCreationError as e:
    print(f"❌ 時間特徵創建失敗: {e.message}")
except Exception as e:
    print(f"❌ 未預期錯誤: {e}")

In [ ]:
# 載入和準備數據
print("🔄 載入數據...")

# 初始化數據處理器和特徵工程器
data_processor = DataProcessor()
feature_engineer = FeatureEngineer()
config = get_config()

try:
    # 載入訓練數據
    df = data_processor.load_data('../train_transaction.csv', '../train_identity.csv')
    print(f"✅ 數據載入成功: {df.shape}")
    
    # 基本數據檢查
    print(f"\n📊 數據概覽:")
    print(f"總交易數: {len(df):,}")
    print(f"詐騙交易: {df['isFraud'].sum():,}")
    print(f"詐騙率: {df['isFraud'].mean():.4f}")
    print(f"特徵數: {df.shape[1]}")
    
except Exception as e:
    print(f"❌ 數據載入失敗: {e}")
    # 如果無法載入，創建示例數據進行演示
    df = pd.DataFrame({
        'TransactionID': range(1000),
        'TransactionDT': np.random.randint(86400, 86400*30, 1000),
        'TransactionAmt': np.random.lognormal(3, 1, 1000),
        'isFraud': np.random.choice([0, 1], 1000, p=[0.965, 0.035]),
        'card1': np.random.randint(1000, 20000, 1000),
        'addr1': np.random.randint(100, 500, 1000)
    })
    print(f"⚠️ 使用示例數據進行演示: {df.shape}")

# 顯示數據前幾行
display(df.head())

In [ ]:
# 導入必要的庫和模組
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import sys
import os

# 添加src目錄到路徑
sys.path.append('../src')

# 導入自定義模組
from feature_engineering import FeatureEngineer
from data_processing import DataProcessor
from config import get_config
from exceptions import FeatureEngineeringError, FeatureCreationError

import warnings
warnings.filterwarnings('ignore')

# 設置視覺化
plt.style.use('default')
sns.set_palette('husl')
plt.rcParams['figure.figsize'] = (12, 8)
pd.set_option('display.max_columns', 100)

print("✅ 所有庫和模組已成功導入！")